In [ ]:
import pandas as pd
import nltk
import string
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from nltk.corpus import stopwords

nltk.download('stopwords')

df = pd.read_excel('whatsapp_appstore.csv')

df.insert(loc=len(df.columns), column='ulasan', value='')

# ------ Proses Tokenisasi ---------

def remove_tweet_special(text):
    text = text.replace('\\t', " ").replace('\\n', " ").replace('\\u', " ").replace('\\', "")
    text = text.encode('ascii', 'replace').decode('ascii')
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\\w+:\\/\\/\\S+)", " ", text).split())
    return text.replace("http://", " ").replace("https://", " ")

def remove_emoticons(text):
    emoticon_pattern = re.compile("["
                                  u"\U0001F600-\U0001F64F"  # emoticons
                                  u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                  u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                  u"\U0001F700-\U0001F77F"  # alchemical symbols
                                  u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                                  u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                                  u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                                  u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                                  u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                                  u"\U00002702-\U000027B0"  # Dingbats
                                  u"\U000024C2-\U0001F251"
                                  "]+", flags=re.UNICODE)
    return emoticon_pattern.sub(r'', text)

def remove_punctuation_except_space(text):
    return re.sub(r'[^\w\s]', '', text)

def remove_whitespace_LT(text):
    return text.strip()

def remove_multiple_whitespace(text):
    return re.sub('\s+', ' ', text)

df['ulasan'] = df['review_description'].fillna('').astype(str)
df['ulasan'] = df['ulasan'].apply(remove_tweet_special).apply(remove_emoticons)
df['ulasan'] = df['ulasan'].apply(remove_punctuation_except_space).apply(remove_whitespace_LT).apply(remove_multiple_whitespace)

stemmer = StemmerFactory().create_stemmer()
factory = StopWordRemoverFactory()
sw = factory.get_stop_words()
if 'tidak' in sw:
    sw.remove('tidak')

list_stopwords = stopwords.words('indonesian')

list_stopwords.extend(['jg', "yg", "dg", "rt", "dgn", "ny", "d", 'klo',
                       'kalo', 'amp', 'biar', 'bikin', 'bilang',
                       'krn', 'nya', 'nih', 'sih',
                       'si', 'tau', 'tuh', 'utk', 'ya',
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 'href','to','the','and','bgt','tp','so','soo','sooo','gw','gua','lu','udh',
                       'udh','ud','pdhl','nih','iya','iyaa','lg','banget','udah','pas','pake','pakai'
                       ,'dpt','kali','terus','trs','trus','is','a','an','oi','woi','pk','pke','deh','jr','ntar','wkwk','dah'
                       ,'dh'])

list_stopwords.extend(sw)
list_stopwords = set(list_stopwords)

def remove_stop(x):
    return " ".join([word for word in str(x).split() if word not in list_stopwords])

df['ulasan'] = df['ulasan'].apply(remove_stop)

df.to_csv('whatsapp_playstore.csv', index=False)

print("Preprocessing selesai. File disimpan sebagai whatsapp_preprocessed.csv")


[nltk_data] Downloading package stopwords to C:\Users\M S
[nltk_data]     I\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Preprocessing selesai. File disimpan sebagai whatsapp_preprocessed.csv
